In [1]:
#Importing necessary dependencies...
import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET
import os
import cv2

from PIL import Image
import albumentations as A

import keras
from tensorflow.keras.models import Model

from tensorflow.keras.layers import (Input, Conv2D,Dropout, Dense, Flatten, BatchNormalization, Reshape, LeakyReLU)
from tensorflow.keras.optimizers import Adam


2024-05-19 16:49:12.204967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 16:49:12.205056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 16:49:12.308940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#list to contain the images to be used in validation set
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

## Initializing variables

In [52]:
train_images='/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages/' #location of images
train_maps='/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/Annotations/'  #location of labels

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2                    #number of bounding box outputed per grid cell
N_CLASSES=len(classes)  
H,W =224,224           #image width, height
SPLIT_SIZE=H//32       #No. of grid cells
print(SPLIT_SIZE)
N_EPOCHS=15
BATCH_SIZE=32

7


## Preprocessing Data

In [9]:
#function to extract bbox,labels from xml file
def preprocess_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    size_tree = root.find('size')
    height = float(size_tree.find('height').text)
    width = float(size_tree.find('width').text)
    bounding_boxes=[]
    for object_tree in root.findall('object'):
        for bounding_box in object_tree.iter('bndbox'):
            xmin = (float(bounding_box.find('xmin').text))
            ymin = (float(bounding_box.find('ymin').text))
            xmax = (float(bounding_box.find('xmax').text))
            ymax = (float(bounding_box.find('ymax').text))
            break
        class_name = object_tree.find('name').text
        class_dict={classes[i]:i for i in range(len(classes))}
        #creating bbox containing normalized image centre coords, normalized width,height and class label
        bounding_box = [     
           (xmin+xmax)/(2*width),(ymin+ymax)/(2*height),(xmax-xmin)/width,
           (ymax-ymin)/height,class_dict[class_name]]  
        bounding_boxes.append(bounding_box)
    return tf.convert_to_tensor(bounding_boxes)

In [10]:
#function to convert bbox to yolo format
def generate_output(bounding_boxes):
    output_label=np.zeros((SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5))
    for b in range(len(bounding_boxes)):
        grid_x=bounding_boxes[...,b,0]*SPLIT_SIZE
        grid_y=bounding_boxes[...,b,1]*SPLIT_SIZE
        i=int(grid_x)
        j=int(grid_y)

        output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
        output_label[i,j,5+int(bounding_boxes[...,b,4])]=1
        #shape = (7,7,25)

    return tf.convert_to_tensor(output_label,tf.float32)

In [13]:
im_paths=[]    #list containing img paths
xml_paths=[]   #list containing img labels(bbox,classes)

val_im_paths=[]  
val_xml_paths=[]


for i in os.listdir(train_maps):
    str = "2007_"+i[:-3]+"jpg"
#     print(str)
    if(str in val_list):
        val_im_paths.append(train_images+i[:-3]+'jpg')
        val_xml_paths.append(train_maps+i)
    else:
        im_paths.append(train_images+i[:-3]+'jpg')
        xml_paths.append(train_maps+i)
print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

9899 9899
64 64


In [14]:
train_dataset=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [15]:
for i in val_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages/000033.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/Annotations/000033.xml'>)


In [16]:
#function to extract img and xml from their paths
def get_imbboxes(im_path,xml_path):
    img=tf.io.decode_jpeg(tf.io.read_file(im_path))
    img=tf.cast(tf.image.resize(img, [H,W]),dtype=tf.float32)

    bboxes=tf.numpy_function(func=preprocess_xml, inp=[xml_path], Tout=tf.float32)
    return img,bboxes

In [17]:
train_dataset=train_dataset.map(get_imbboxes)
val_dataset=val_dataset.map(get_imbboxes)

In [18]:
for i,j in train_dataset.skip(2):
    print(i.shape,j)
    break

(224, 224, 3) tf.Tensor([[0.47       0.35866666 0.128      0.14666666 2.        ]], shape=(1, 5), dtype=float32)


In [19]:
#Implementing Augementation pipeline to prevent overfitting
transforms = A.Compose([
    A.Resize(H,W),
    A.RandomCrop(
         width=np.random.randint(int(0.9*W),W),
         height=np.random.randint(int(0.9*H),H), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(H,W),
    
], bbox_params=A.BboxParams(format='yolo', ))

In [20]:
def aug_albument(image,bboxes):
    augmented=transforms(image=image,bboxes=bboxes)
    return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]

In [21]:
def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [22]:
train_dataset=train_dataset.map(process_data)

In [23]:
for i,j in train_dataset.skip(2):
    print(i.shape,j)
    break

(224, 224, 3) tf.Tensor([[0.47       0.35866666 0.128      0.14666666 2.        ]], shape=(1, 5), dtype=float32)


In [24]:
def preprocess_augment(img,y):
    img = tf.image.random_brightness(img, max_delta=50.)
    img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
    img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
    img = tf.clip_by_value(img, 0, 255)
    labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
    return img,labels

In [25]:
def preprocess(img,y):
    img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

    labels=tf.numpy_function(func=generate_output, inp=[y], Tout=(tf.float32))
    return img,labels

In [26]:
train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [27]:
NUM_FILTERS=512   #Number of filters in conv layer
OUTPUT_DIM=N_CLASSES+5*B  #Last dimension of the output of conv layer

In [28]:
#function to assert shapes of images and labels
def _fixup_shape(images, labels):
    images.set_shape([H,W,3])
    labels.set_shape([SPLIT_SIZE,SPLIT_SIZE,25])
    return images, labels

In [29]:
train_dataset = train_dataset.map(_fixup_shape).batch(BATCH_SIZE)
val_dataset = val_dataset.map(_fixup_shape).batch(BATCH_SIZE)

In [30]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 7, 25), dtype=tf.float32, name=None))>

## Model 

In [32]:
def model(input_shape=(224, 224, 3),weights='imagenet'):
    base_model = tf.keras.applications.VGG16(weights=weights, include_top=False, input_shape=input_shape)

    # Freeze the pre-trained layers
    for layer in base_model.layers:   #setting layers in VGG to be non-trainable
        layer.trainable = False

    # Add custom top layers
    x = base_model.output

    x = Conv2D(NUM_FILTERS,(3,3), padding = 'same',kernel_initializer='he_normal',)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)

    x = Conv2D(NUM_FILTERS,(3,3), padding = 'same',kernel_initializer='he_normal',)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)

    x = Flatten(name='flatten')(x)

    x = Dense(NUM_FILTERS,kernel_initializer='he_normal',)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)

    x = Dropout(0.5)(x)

    x = Dense(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIM,activation='sigmoid')(x)

    outputs = Reshape((SPLIT_SIZE,SPLIT_SIZE,OUTPUT_DIM))(x)

    # Create model
    model = Model(inputs=base_model.input, outputs=outputs, name='YOLO')

    return model

In [33]:
model = model(input_shape=(224,224,3))
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "YOLO"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 33,040,126 (126.04 MB)

 Trainable params: 18,322,366 (69.89 MB)

 Non-trainable params: 14,717,760 (56.14 MB)

In [43]:
#IOU calculation
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0], axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0], axis=-1)

    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [44]:
def difference(x,y):
    return tf.reduce_sum(tf.square(y-x))

<img src="image-2.png" alt="Loss" width="800" height="400">

In [45]:
#implementing the YOLO loss function
def yolo_loss(y_true, y_pred):
    target = y_true[..., 0]
    ## Object Loss
    non_zero_indices = tf.where(tf.equal(target, 1))
    y_pred_extract = tf.gather_nd(y_pred, non_zero_indices)
    y_target_extract = tf.gather_nd(y_true, non_zero_indices)
    
    rescaler = tf.cast(non_zero_indices * 32, tf.int32)
    rescaler_shape_0 = tf.shape(rescaler)[0]
    
    upscaler_1 = tf.concat([rescaler[:, 1:], tf.zeros([rescaler_shape_0, 2], dtype=tf.int32)], axis=-1)
    
    upscale_factors = tf.constant([32.0, 32.0, 224, 224], dtype=tf.float32)
    upscale_factors = tf.reshape(upscale_factors, [1, 4])
    
    target_upscaler_2 = upscale_factors * y_target_extract[..., 1:5]
    pred_1_upscaler_2 = upscale_factors * y_pred_extract[..., 1:5]
    pred_2_upscaler_2 = upscale_factors * y_pred_extract[..., 6:10]
    
    target_orig = tf.cast(upscaler_1, dtype=tf.float32) + target_upscaler_2
    pred_1_orig = tf.cast(upscaler_1, dtype=tf.float32) + pred_1_upscaler_2
    pred_2_orig = tf.cast(upscaler_1, dtype=tf.float32) + pred_2_upscaler_2
    
    iou_pred_1 = compute_iou(target_orig, pred_1_orig)
    iou_pred_2 = compute_iou(target_orig, pred_2_orig)
    
    mask = tf.cast(iou_pred_2 > iou_pred_1, dtype=tf.int32)
    
    y_pred_joined = tf.stack([y_pred_extract[..., 0], y_pred_extract[..., 5]], axis=-1)
    obj_pred = tf.gather(y_pred_joined, mask, batch_dims=1)
    
    object_loss = difference(obj_pred, tf.ones_like(obj_pred))
    ## Non-object loss
    zero_indices = tf.where(tf.equal(target, 0))
    y_pred_zero = tf.gather_nd(y_pred[..., :10], zero_indices)
    no_object_loss_1 = difference(y_pred_zero[..., 0], tf.zeros_like(y_pred_zero[..., 0]))
    no_object_loss_2 = difference(y_pred_zero[..., 5], tf.zeros_like(y_pred_zero[..., 5]))
    
    no_object_loss = no_object_loss_1 + no_object_loss_2
    ## NO object loss
    class_pred = tf.gather_nd(y_pred[..., 10:], non_zero_indices)
    class_true = tf.gather_nd(y_true[..., 5:], non_zero_indices)
    
    class_loss = difference(class_pred, class_true)
    ## Object bounding box loss
    centres_pred = tf.stack([y_pred_extract[..., 1:3], y_pred_extract[..., 6:8]], axis=1)
    centres_pred = tf.gather(centres_pred, mask, batch_dims=1)
    centres_true = y_target_extract[..., 1:3]
    
    centre_loss = difference(centres_pred, centres_true)
    
    sizes_pred = tf.stack([y_pred_extract[..., 3:5], y_pred_extract[..., 8:10]], axis=1)
    sizes_pred = tf.gather(sizes_pred, mask, batch_dims=1)
    sizes_true = y_target_extract[..., 3:5]
    
    size_loss = difference(tf.math.sqrt(tf.abs(sizes_pred)), tf.math.sqrt(tf.abs(sizes_true)))
    box_loss = centre_loss + size_loss
    
    lambda_coord = 5.0
    lambda_no_obj = 0.5
    
    loss = object_loss + (lambda_no_obj * no_object_loss) + (lambda_coord * box_loss) + class_loss
    
    return loss

In [46]:
model.compile(
  loss=yolo_loss,
  optimizer=Adam(1e-3), 
)

In [53]:
history = model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=N_EPOCHS,
)

Epoch 1/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 115s 372ms/step - loss: 142.2339 - val_loss: 150.4526
Epoch 2/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 115s 372ms/step - loss: 140.9671 - val_loss: 149.0675
Epoch 3/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 115s 371ms/step - loss: 137.9303 - val_loss: 146.4022
Epoch 4/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 116s 375ms/step - loss: 135.7340 - val_loss: 147.2779
Epoch 5/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 115s 370ms/step - loss: 133.1351 - val_loss: 145.4206
Epoch 6/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 115s 370ms/step - loss: 130.1217 - val_loss: 143.4196
Epoch 7/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 114s 369ms/step - loss: 126.5541 - val_loss: 144.3303
Epoch 8/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 116s 374ms/step - loss: 125.4681 - val_loss: 142.6570
Epoch 9/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 114s 369ms/step - loss: 123.6094 - val_loss: 146.9426
Epoch 10/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 114s 367ms/step - loss: 121.4320 - val_loss: 139.9440
Epoch 11/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 114s 366ms/step - 